In [1]:
import pandas as pd
import openai
import json
import time
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [2]:
#pip install openai

In [3]:
from openai import OpenAI

In [4]:
openai.__version__


'1.1.1'

### Read in annotated data

In [5]:
#df = pd.read_csv("../data/data_splits_stratified/6-2-2_all_classes_enriched_with_kw/test.csv", index_col=0)
df = pd.read_csv("../data/data_splits_stratified/6-2-2_all_classes_enriched/test.csv", index_col=0)

In [6]:
df.shape

(534, 7)

In [7]:
df.head()

,pmid,journal_name,title,abstract,accepted_label,multi_label,binary_label
idx,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,Non-systematic-review,1,0
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Non-systematic-review,1,0
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,Non-systematic-review,1,0
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,Non-systematic-review,1,0
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,Non-systematic-review,1,0


In [8]:
# Combine the columns
# Implementing custom tags for the combination of journal name, title, and abstract
df['input_journal_title_abstract'] = '<journal>' + df['journal_name'] + '</journal>' + \
                                     '<title>' + df['title'] + '</title>' + \
                                     '<abstract>' + df['abstract'] + '</abstract>'

# Implementing custom tags for the combination of title and abstract only
df['input_title_abstract'] = '<title>' + df['title'] + '</title>' + \
                             '<abstract>' + df['abstract'] + '</abstract>'

In [9]:
df.head(5)

,pmid,journal_name,title,abstract,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract
idx,,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...,<title>Can clinical practice guide a research ...
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...,<title>Dual/multitargeted xanthone derivatives...
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...,<title>Benefits of occupational therapy in str...
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...,<title>The promise of ketamine for treatment-r...
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...,<title>[Clinical and pathogenic aspects of sec...


### Load key for the OpenAI API 

In [10]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

Note: You need to create a credentials.txt file with the following content:  
OPENAI=sk-77QXXXXXXXXXXXXXXXXXXXXXXXXXXX  
replace the value after the = sign with your API key.  
Make sure the credentials.txt is added to .gitignore, you don't want to put your password on Git!

In [11]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")


Found password.


In [12]:
client = OpenAI(api_key=openai.api_key)

### Query GPT

To change the task the model is solving, you need to change the text of the prompt and the content text of the system role.  
To change the GPT model used, you need to change the text of the model name when initiating the openai API.  
The function gets as input the input_raw_text, that will be text for information extraction or classification.

In [13]:
import time

DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

def query_gpt(input_raw_text, prompt_text, gpt_model="gpt-3.5-turbo", temperature=0, max_retries=5, retry_delay=3):
    # CHANGE gpt_model to the desired model name, see https://platform.openai.com/docs/models (gpt-3.5-turbo and gpt-4-turbo-preview)
    
    # Add a delay at the beginning of the function to avoid overloading the API if there are multiple calls
    # time.sleep(10)  

    system_msg = f"""
    You are an expert assistant specialized in text classification of PubMed abstracts. """

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                #max_tokens=2000,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text + input_raw_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            # Handle API error, e.g., retry or log
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)  # Wait before retrying
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")


In [14]:
# Define a function to apply GPT queries with a progress bar
def apply_gpt_with_progress(data_series, prompt_text, model="gpt-3.5-turbo"):
    results = []
    total_items = len(data_series)
    # Create a tqdm progress bar
    with tqdm(total=total_items, desc=f"Processing dataset") as pbar:
        for text in data_series:
            result = query_gpt(text, prompt_text, model)
            results.append(result)
            pbar.update(1)  # Update the progress bar

    return results

# Multi-label: Read prompts from file and query GPT

In [15]:
json_file_path = "./prompt_strategies_shirin.json"
# Load the JSON file
with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [16]:
df.head() #show first few lines of df
#print(df) #shows whole df

,pmid,journal_name,title,abstract,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract
idx,,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...,<title>Can clinical practice guide a research ...
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...,<title>Dual/multitargeted xanthone derivatives...
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...,<title>Benefits of occupational therapy in str...
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...,<title>The promise of ketamine for treatment-r...
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...,<title>[Clinical and pathogenic aspects of sec...


In [17]:
## Sample to test and demo
#sampled_df = df.sample(n=50, random_state=1)
#sampled_df.head()

## Run different prompts over the data

In [18]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P11_3", "P11_4", "P11_5", "P12"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df[f'gpt_predictions_{prompt_id}'] = df[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        #df.to_csv(f"predictions/{model}_enriched_kw_test_outputs_{'_'.join(prompt_ids_to_test)}.csv")
        df.to_csv(f"predictions/{model}_enriched_test_outputs_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")
        

Skipping prompt P1
Skipping prompt P2
Skipping prompt P2_1
Skipping prompt P3
Skipping prompt P3_1
Skipping prompt P3_2
Skipping prompt P3_3
Skipping prompt P3_4
Skipping prompt P4
Skipping prompt P4_1
Skipping prompt P4_2
Skipping prompt P5
Skipping prompt P6
Skipping prompt P7
Skipping prompt P8
Skipping prompt P8_1_1
Skipping prompt P8_1_2
Skipping prompt P8_1_3
Skipping prompt P9
Skipping prompt P9_1
Skipping prompt P10


Processing dataset:   0%|                               | 0/534 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   0%|                       | 1/534 [00:01<09:38,  1.09s/it]

Trying to call OpenAI API...


...

Trying to call OpenAI API...


Processing dataset: 100%|████████████████████▉| 533/534 [06:47<00:00,  1.30it/s]

Trying to call OpenAI API...


Processing dataset: 100%|█████████████████████| 534/534 [06:48<00:00,  1.31it/s]

Skipping prompt P11_3
Skipping prompt P11_4
Skipping prompt P11_5
Skipping prompt P12


In [19]:
df.head()

,pmid,journal_name,title,abstract,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract,gpt_predictions_P11_raw,gpt_predictions_P11,gpt_predictions_P11_1_raw,gpt_predictions_P11_1,gpt_predictions_P11_2_raw,gpt_predictions_P11_2
idx,,,,,,,,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...,<title>Can clinical practice guide a research ...,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...,<title>Dual/multitargeted xanthone derivatives...,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...,<title>Benefits of occupational therapy in str...,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention,"{\n ""gpt_label"": ""Remaining""\n}",Remaining,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...,<title>The promise of ketamine for treatment-r...,"{\n ""gpt_label"": ""Human-non-RCT-drug-interv...",Human-non-RCT-drug-intervention,"{\n ""gpt_label"": ""Human-non-RCT-drug-interv...",Human-non-RCT-drug-intervention,"{\n ""gpt_label"": ""Human-non-RCT-drug-interv...",Human-non-RCT-drug-intervention
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...,<title>[Clinical and pathogenic aspects of sec...,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review,"{\n ""gpt_label"": ""Non-systematic review""\n}",Non-systematic review
